In [ ]:
# automatically reload modules if source is modified 
%load_ext autoreload
%autoreload 2

In [ ]:
import chaospy as cp
import easyvvuq as vvuq
import enum 
import os
import pathlib
import yaml

In [ ]:
from isct.trial import Trial 

In [ ]:
# the path where Singularity containers are stored
# provide `None` to use Docker
container_path = None
# container_path = pathlib.Path('/containers')

In [ ]:
# path for a reference trial and the VVUQ database(s)
trial_dir = pathlib.Path('/Users/max/trials/singleton')
datab_dir = pathlib.Path('/Users/max/trials/vvuq')

# the template points to the first patient directory
template_dir = pathlib.Path('/Users/max/trials/singleton/patient_00000')
template_dir

In [ ]:
def sub(path):
    subdir = map(lambda p: os.path.join(path, p), os.listdir(path))
    return list(filter(os.path.isdir, subdir))

def dir_to_dict(dir):
    if sub(dir) == []: 
        return None
    return {os.path.basename(p): dir_to_dict(p) for p in sub(dir)}

In [ ]:
# the directory encoder
tree = dir_to_dict(template_dir)
dir_enc = vvuq.encoders.DirectoryBuilder(tree=tree)
dir_enc

In [ ]:
# the YAML encoder (still a custom class)
# TODO: might want to submit a pull request for this encoder?
from easyvvuq.encoders import BaseEncoder
import yaml
class YAMLEncoder(BaseEncoder, encoder_name='yaml_encoder'):
    def __init__(self, template_filename, target_filename=None):
        self.template_filename = str(template_filename)
        if target_filename:
            self.target_filename = target_filename
        else:
            self.target_filename = os.path.basename(self.template_filename)
        
    def encode(self, params={}, target_dir=''):
        if not target_dir: 
            raise RuntimeError('No target directory specified to encode')
            
        try:
            with open(self.template_filename, 'r') as template_file:
                yaml_file = yaml.safe_load(template_file)
        except FileNotFoundError:
            raise RuntimeError(
            f'the template file specified ({self.template_filename}) does not exist')
        
        # Extract parameters that are actually present in the
        # configuration file already. These are the subset of
        # parameters that we update during encoding
        matched_params = {k:v for (k, v) in params.items() if k in yaml_file}
            
        yaml_file.update(matched_params)
        target_file_path = os.path.join(target_dir, self.target_filename)
        with open(target_file_path, 'w') as fp:
            yaml.dump(yaml_file, fp)
            
    def get_restart_dict(self):
        return {"target_filename": self.target_filename,
                "template_filename": self.template_filename}
    
    def element_version(self):
        return "0.1"

In [ ]:
# The blood flow is not a common file format, it is given 
# as a `key=value\n` text file. We can cook up a specific
# class to handle these cases, but we can also do a little 
# trick..., we can construct a template from the parameters
# we want to modify

# and then, we use that template file with the already
# provided generic encoders

def bf_processor(original_fn, template_fn, params):
    # process line by line, replace `key=value` by `key=$key`
    with open(template_fn, 'w') as template:
        with open(original_fn, 'r') as original:
            for line in original:
                key, value = line.strip().split('=')
                if key in params:
                    # replace by template if present in parameters
                    value = f'${key}'
                template.write(f'{key}={value}\n')

def create_generic_encoder(processor, params, template_dir, filepath, delimiter='$'):
    from easyvvuq.encoders import GenericEncoder
    
    # setup the input/output paths
    original_fn = template_dir.joinpath(filepath)
    base, _ = os.path.splitext(filepath)  # split original extension
    template_fn = template_dir.joinpath(f'{base}.template')
    
    # evaluate the processor function
    processor(original_fn, template_fn, params)
    
    # setup the encoder 
    encoder = GenericEncoder(str(template_fn), delimiter=delimiter, target_filename=str(filepath))
    return encoder
    
bf_enc = create_generic_encoder(bf_processor, {'BLOOD_VISC', 'Density'}, template_dir, 'baseline/bf_sim/Model_parameters.txt')     
bf_enc

In [ ]:
# the patient configuration encoder
patient_enc = YAMLEncoder(template_dir.joinpath('patient.yml'))

This table lists a copy of the considered variables for UQ analysis of the one-dimensional bloodflow model. Each variable should be updated through `EasyVVUQ` and requires Encoders/Decoders to do so. The column `supported` indicates whether this is possible yes or no. The file names are all relative to the patient directory, i.e `trial/patient_i`.

These are all _inputs_ towards the bloodflow model, where various outputs are possible, e.g. the flow or related properties for a variety of arteries. TODO: what are the output fields of interest, i.e. which artery and what physical property?

| variable `name` | type | unit | location | supported | range | 
| ------ | ------ | ------ | ------ | ----- | ----- | 
| heart rate `HeartRate` | uncertain | bmp | `patient.yml`, `config.xml` | yes | N(68,20) | 
| stroke volume `StrokeVolume` | uncertain | ml | `bf_sim/Model_parameters.txt` | yes | N(104,21) | 
| blood density `Density` | uncertain | kg.m-3 | `bf_sim/Model_parameters.txt` | yes | U(1019,1061)|
| blood viscosity `BLOOD_VISC` | uncertain | mPa.s | `bf_sim/Model_parameters.txt` | yes | N(62.9,18.1) |
| wall thickness | uncertain | mm | per vessel: `1-D_Anatomy.txt` | no | N(0.44,0.04) |
| wall elasticity | uncertain | mmHg | per vessel: `1-D_Anatomy.txt` | no | N(951,380) |
| vertebral artery diameter | uncertain | mm | `unknown` | no | U(3.2,6.5) |
| systolic pressure `SystolePressure` | certain | mmHg | `bf_sim/Model_parameters.txt` | yes | kept to value from WP2 (`rr_syst`) |
| diastolic pressure `DiastolePressure` | certain | mmHg | `bf_sim/Model_parameters.txt` | yes | kept to hardcoded default value as in `workflow/patient.py` |
| mean right atrial pressure | certain | mmHg | `unknown` | no | |  
| clot location | certain | categorical | `Clots.txt` | no | kept to hardcoded value `R. MCA` as in `workflow/patient.py` until issue #48 is resolved. Should be considered as discrete distribution |
| CoW vessel diameters | certain | mm | `unknown` | no | |
| CoW vessel lengths | certain | mm | `unknown` | no | |
| brain mesh | certain | mm | `unknown` | no | |

TODO:
- Unclear if yet possible to switch brain meshes.
- Many data on vessel diamaters: which to vary for UQ?
- Elasticity appears in `1-D_Anatomy.txt` for each vessel: which to vary for UQ?

In [ ]:
# create a `campaign`: effectively a directory containing a SQL database
# each campain is formed by the prefix with a unique identifier as postfix
prefix = 'UQ_'
campaign = vvuq.Campaign(prefix, work_dir=datab_dir)

In [ ]:
files_to_copy = [
    template_dir.joinpath('Clots.txt'),
    template_dir.joinpath('baseline/1-D_Anatomy.txt'),
    template_dir.joinpath('baseline/CoW_Complete.txt'),
    template_dir.joinpath('baseline/1-D_Anatomy_Patient.txt'),
]

for path in os.listdir(template_dir.joinpath('baseline')):
    path = template_dir.joinpath('baseline').joinpath(path)
    if os.path.isfile(path):
        files_to_copy.append(path)      

targets = [p.relative_to(template_dir) for p in files_to_copy]
copy_enc = [vvuq.encoders.CopyEncoder(str(src), str(dst)) for src, dst in zip(files_to_copy, targets)]

targets

In [ ]:
encoder = vvuq.encoders.MultiEncoder(*[dir_enc, *copy_enc, patient_enc, bf_enc])
encoder

In [ ]:
# output file where `pressure drop` data is written to 
# TODO: this should probably be provided somewhere as constants
output = "baseline/bf_sim/ResultsPerVessel.csv"

# output variables of interest in `ResultPerVessel.csv`
cols = ["VolumeFlowrate(mL/s)", "Pressure(Pa)"]

# create a `decoder`: decode the output parameters towards the database
decoder = vvuq.decoders.SimpleCSV(target_filename=output, output_columns=cols)

In [ ]:
# define parameters of interest and their properties
# this all just goes into a single dictionary, where now only 
# `BLOOD_VISC` is considered as parameter to be varied 
parameters = {
    #"HeartRate": { 
    #    "type": "float", 
    #    "min": 0,
    #    "max": 200,
    #    "default": 73,
    #},
    "StrokeVolume": {
        "type": "float",
        "min": 0,
        "max": 250,
        "default": 95,
    },
    "Density": {
        "type": "float",
        "min": 0.0,
        "max": 3000,
        "default": 1050,
    },
    "BLOOD_VISC": {
        "type": "float", 
       "min": 0.0, 
        "max": 10.0e-3, 
    "default": 4.2e-3,
    }
}

In [ ]:
# create an `app` for the campaign, by connecting all components
campaign.add_app(
    name="blood-visc",
    params=parameters,
    encoder=encoder,
    decoder=decoder,)

### Sampling definition

In [ ]:
# the parameters to vary are provided as dict with their 
# corresponding distributions 
vary = {
    #"HeartRate": cp.Normal(73, 12.2), 
    "StrokeVolume": cp.Normal(95, 14),
    "Density": cp.Uniform(1040,1055),
    "BLOOD_VISC": cp.Normal(4.2e-3, 0.9e-3),
}

In [ ]:
# available methods
class Method(enum.Enum):
    random = "random"
    PCE = "PCE"
    QMC = "QMC"

In [ ]:
# pick any from the Enum 
method = Method.QMC

# create a `sampler` matching the method 
if method == method.random: 
    sampler = vvuq.sampling.RandomSampler(vary=vary)
    
if method == method.PCE: 
    sampler = vvuq.sampling.PCESampler(vary=vary, polynomial_order=3)
    
if method == method.QMC:
    sampler = vvuq.sampling.QMCSampler(vary=vary, n_mc_samples=10**4) # this is the default

# assign the sampler
campaign.set_sampler(sampler)

The `num_samples` variable seems to act as either a limit or indication of the desired number of samples to be drawn. For the more advanced methods, PCE and QMC, it seems most logical to set the number of samples sufficiently high, such that PCE/QMK can dictate the required number of samples to draw. Note, if PCE/QMC are restricted to too little samples, the corresponding analysis might not be able to execute.

In [ ]:
# draw the samples
num_samples = 500
replicas = 1 # the number of times a single sample is replicated
campaign.draw_samples(num_samples=num_samples, replicas=replicas)

In [ ]:
# This logs all the runs in the current `campaign`. It is mostly for 
# simple inspection to see if the desired parameters are varied and to 
# list all runs. 
for run in campaign.list_runs():
    print(f"{run[1]['run_name']}: {run[1]['params']}")

In [ ]:
# create all run directories; copies the template and updates the
# parameters using the `ISCTEncoder`
campaign.populate_runs_dir()
state = datab_dir.joinpath("state.json")
campaign.save_state(state)

In [ ]:
run_dir = campaign.db_location.split(":")[-1]
run_dir = pathlib.Path(run_dir).parent
run_dir

In [ ]:
with open(run_dir.joinpath('runs/trial.yml'), 'w') as config_file:
    trial_config = {
        'container-path': '/scratch/containers'
    }
    yaml.dump(trial_config, config_file)

In [ ]:
for (run, _) in campaign.list_runs():
    print(run)

In [ ]:
dir(campaign)

### Running `isct` for each proposed sample

By populating the campaign, all required subdirectories are created in the directory of the database. These directories represent patient directories for which various ways are available to evaluate their simulations. We can consider the base directory as a trial directory, and invoke `isct trial run` to evaluate the simulations of all subdirectories. Alternatively, and more involved, we could manually invoke the individual runs by `isct patient run`. Eitherway, the required steps are evaluated and the output is stored within the individual run directories. Afterwards, the collation step will aggregate these results back into the database. 

To run the jobs in parallel, we can do so locally by exploiting parallel with `n` procs (`-jn`)

`isct trial run {run_dir} --gnu-parallel | parallel -jn`

TODO:
- support running notebook on external workstation: evacuate jobs locally on the remote machine
- support remote execution on workstations: send the jobs towards the remote workstation for execution
- support remote execution on HPC systems: send the jobs through a queing system to HPC systems 
- investigate efficient collation: archive data sets remotely, transport only essential information for analysis

In [ ]:
# extract the path from the database location, this seems required
# to obtain the hash that is attached after the original directory 
run_dir = campaign.db_location.split(":")[-1]
run_dir = pathlib.Path(run_dir).parent

# the runs are located in the /runs/ directory
run_dir = run_dir.joinpath("runs")

In [ ]:
# two approaches to running the trial

# 1. from the cmd-line manually 
# evaluate with `-x` to log the commands, without `-x` to
# actually run the commands
print(f"isct -v trial run {run_dir} -x --clean-files")

# 2. from the notebook directly
# select the `Logger()` to log the commands to output
# select the `LocalRunner()` to run the commands locally
# this evaluates the run commands per patient in a subshell
#from isct.runner import Logger, LocalRunner
#runner = Logger()
# runner = LocalRunner()
#keep_files = False  # remove large output files

#for patient in Trial(path=run_dir, runner=runner, keep_files=keep_files):
#    patient.run()

### Collecting output
This steps collects the output parameters of interest from the output files and stores the output in the database.

In [ ]:
campaign.collate()

### Analysing output
- The analysis has to match the sampling method, these are directly related.
- The analysis seems to fail for PCE/QMC when too little samples are considered.

In [ ]:
# define analysis in line with sampling method
if method == method.random:
    analysis = vvuq.analysis.BasicStats(
        qoi_cols=cols
    )

if method == method.PCE: 
    analysis = vvuq.analysis.PCEAnalysis(
        sampler=sampler, 
        qoi_cols=cols
    )
    
if method == method.QMC:
    analysis = vvuq.analysis.QMCAnalysis(
        sampler=sampler,
        qoi_cols=cols
    )

In [ ]:
# apply analysis to current database
campaign.apply_analysis(analysis)
results = campaign.get_last_analysis()

### Results
The results are reported as a dictionary, where the contents are strongly dependent on the chosen sampling and analysis method. Both PCE and QMC report Sobol indices in addition to basic statistical information. From here, we can either perform the post-processing directly on the obtained dictionary, however, direct data analysis on the database is also a possibility. Note, it seems to make sense to exploit the already provided analysis methods provided in EasyVVUQ as much as possible.

In [ ]:
# The dictionary keys depend on the analysis
results.keys()

In [ ]:
# The information obtained from the collation can be accessed explicitly. 
# This returns a panda dataframe with all the data fields, this can be
# used for any type of analysis as well.
hr = campaign.get_collation_result()[4::225]['Pressure(Pa)']

### Storing and saving results
The state of the campaign can be stored explicitly and from there reloaded elsewhere. This should provided the necessary functionality to serialise the current state of the campaign, together with the database and all subdirectories, and move this archive between systems. Thus, we can move around the database and the runs of all samples between systems, e.g. between remote and local machines.

The `state.json` simply contains the type of samplers, collation, and aggregation methods as well as the details of the database, i.e. its path, and the working directory that contains all subdirectories of the the individual samples. From there, we can initialise a new campaign and continue where previously left of. Thus, the data analysis could be decoupled completely from the scripts that perform the VVUQ analysis, which also saves recomputation compared to reevaluating the database over and over.

In [ ]:
from easyvvuq.decoders import YAMLDecoder

class ISCTDecoder(YAMLDecoder, decoder_name="ISCTDecoder"):
    """ISCTDecoder decodes the patient outcome from isct trials.

    Currently the implementation wraps a `YAMLDecoder` to extract the patient
    outcome from the `patient_outcome.yml` generated by the `patient-outcome`
    module.
    """
    def __init__(self,
                 target_filename='patient_outcome.yml',
                 output_columns=None):
        super().__init__(target_filename, output_columns)

    def sim_complete(self, run_info=None):
        """Returns True if the target file is present.

        When considering `patient_outcome.yml` as target file, the result of
        sim_complete coincides with an indication if the simulation was
        complete, as the file is only present afterwards.
        """
        assert run_info is not None, "No run information provided."
        return os.patoh.isfile(
            pathlib.Path(run_info['run_dir']).joinpath(self.target_filename))

In [ ]:
state = datab_dir.joinpath("state.json")
campaign.save_state(state)

In [ ]:
state = datab_dir.joinpath("state.json")
reloaded_campaign = vvuq.Campaign(state_file=state, work_dir=datab_dir)
reloaded_campaign.get_collation_result()

In [ ]:
# define a decoder that retrieves the result after reloading the database
# this specifically decodes the output of the blood-flow model 
decoder = vvuq.decoders.SimpleCSV(target_filename=output, output_columns=cols)
run = run_dir.joinpath('Run_1')
decoder.parse_sim_output({'run_dir':run})

In [ ]:
# similarly we can define a decoder for the patient outcome, this can be done 
# using the ISCTDecoder, a wrapper around a YAMLDecoder, that extracts the 
# data from all `patient_outcome.yml` files for the different runs in the 
# database results
output = "patient_outcome.yml"

# accessing output columns of interest from the YAML format can be achieved by 
# providing the root of the variable, while for nested dictionaries, we can 
# provide the comlete tree [root, node, ..., node, leaf] that is traversed,
# where the value of the resuling dictionary [root][node][...][leaf] is 
# extracted and returned in the output dataframe
cols = ["sex", "age", ["thrombectomy", "result"], ['infarct-volume', 'stroke', '11', 'volume']]
decoder = ISCTDecoder(target_filename=output, output_columns=cols)
decoder.parse_sim_output({'run_dir':run})